# Sonification of Bleeding with Bank of Filters

First Meeting (2019-06-11, Sasan and Thomas) @CITEC, TH proposed Filter-bank for feature generation
* The idea is to use a bank of different low-pass filters to create increasingly smooth signals
* these filtered signals serve as source for identifying key moments to anchor sound events
* which then create a multiscale data-driven complex grain structure of the raw instantaneous bleeding data.
* note that the limit of filtering with a cutoff-frequency towards 0 yields the integrated signal.

## Imports

In [ ]:
from scipy import signal
import numpy as np
import scipy.interpolate
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
import copy

In [ ]:
# %matplotlib inline

## Load Data and Create Filter-Bank Signals

In [ ]:
df = pd.read_csv('log_refactored_correction_factor.csv', na_values=['no info', '.'], delimiter=',')
df_indexed = df.reset_index(drop=False)
index = df_indexed['index']
delta = df_indexed['Delta']
volume = df_indexed['Blood Accumulated']

delta_min = delta.min()
delta_max = delta.max()

volume_min = volume.min()
volume_max = volume.max()

print("dataset loaded:")
print(f"  delta:   min={delta_min:8}, max={delta_max:8.3}")
print(f"  volume:  min={volume_min:8}, max={volume_max:8}")

## Event-based Sonification of filtered data (min/max/threshold cut-throughs...)

In [ ]:
import sc3nb as scn
import time
sc = scn.startup()

In [ ]:
#%sc FreqScope(400, 300)
%sc s.makeGui
%sc s.scope

In [ ]:
%%scv
SynthDef("pb-sasan", {|out=0, bufnum=0, rate=1, pan=0, amp=0.3, rel=0.1, dur=0.2|
    //var sig1 = PlayBuf.ar(1, bufnum, 1.01*rate * BufRateScale.kr(bufnum), doneAction: 2);
    //var sig2 = PlayBuf.ar(1, bufnum, 0.99*rate * BufRateScale.kr(bufnum), doneAction: 2);

    var drate = SinOsc.ar(3, add:rate * BufRateScale.kr(bufnum), mul:0.01);
    var sig = PlayBuf.ar(1, bufnum, drate, doneAction: 2);

    var env = EnvGen.kr(Env.new([1,1,0], [dur-rel, rel]), doneAction: 2);
    Out.ar(out, Pan2.ar(sig, pan, amp*env))
}).add();

SynthDef(\bpfsaw, {
    arg atk=2, sus=0, rel=3, c1=1, c2=(-1),
    freq=500, detune=0.2, pan=0, cfhzmin=0.1, cfhzmax=0.3,
    cfmin=500, cfmax=2000, rqmin=0.1, rqmax=0.2,
    lsf=200, ldb=0, amp=1, out=0;
    var sig, env;
    env = EnvGen.kr(Env([0,1,1,0],[atk,sus,rel],[c1,0,c2]),doneAction:2);
    sig = Saw.ar(freq * {LFNoise1.kr(0.5,detune).midiratio}!2);
    sig = BPF.ar(
        sig,
        {LFNoise1.kr(
            LFNoise1.kr(4).exprange(cfhzmin,cfhzmax)
        ).exprange(cfmin,cfmax)}!2,
        {LFNoise1.kr(0.1).exprange(rqmin,rqmax)}!2
    );
    sig = BLowShelf.ar(sig, lsf, 0.5, ldb);
    sig = Balance2.ar(sig[0], sig[1], pan);
    sig = sig * env * amp;
    Out.ar(out, sig);
}).add;

SynthDef(\bpfsine, {
    arg atk=2, sus=0, rel=3, c1=1, c2=(-1),
    freq=500, detune=0.2, pan=0, cfhzmin=0.1, cfhzmax=0.3,
    cfmin=500, cfmax=2000, rqmin=0.1, rqmax=0.2,
    lsf=200, ldb=0, amp=1, out=0;
    var sig, env;
    env = EnvGen.kr(Env([0,1,1,0],[atk,sus,rel],[c1,0,c2]),doneAction:2);
    sig = SinOsc.ar(freq * {LFNoise1.kr(0.5,detune).midiratio}!2);
    sig = BPF.ar(
        sig,
        {LFNoise1.kr(
            LFNoise1.kr(4).exprange(cfhzmin,cfhzmax)
        ).exprange(cfmin,cfmax)}!2,
        {LFNoise1.kr(0.1).exprange(rqmin,rqmax)}!2
    );
    sig = BLowShelf.ar(sig, lsf, 0.5, ldb);
    sig = Balance2.ar(sig[0], sig[1], pan);
    sig = sig * env * amp;
    Out.ar(out, sig);
}).add;

SynthDef(\reverb, {
    arg in, predelay=0.1, revtime=1.8,
    lpf=4500, mix=0.15, amp=1, out=0;
    var dry, wet, temp, sig;
    dry = In.ar(in,2);
    temp = In.ar(in,2);
    wet = 0;
    temp = DelayN.ar(temp, 0,2, predelay);
    16.do{
        temp = AllpassN.ar(temp, 0.05, {Rand(0.001,0.05)}!2, revtime);
        temp = LPF.ar(temp, lpf);
        wet = wet + temp;
    };
    sig = XFade2.ar(dry, wet, mix*2-1, amp);
    Out.ar(out, sig);
}).add;

"SynthDefs loaded".postln;

In [ ]:
%%scv
~bus = Dictionary.new;
~bus.add(\reverb -> Bus.audio(s,2));
"Bus loaded".postln;

In [ ]:
%%scv
~out = 0;
~mainGroup = Group.new;
~reverbGroup = Group.after(~mainGroup);
~reverbSynth = Synth.new(\reverb, [
        \amp, 1,
        \predelay, 0.4,
        \revtime, 1.8,
        \lpf, 4500,
        \mix, 0.5,
        \in, ~bus[\reverb],
        \out, ~out,
    ], ~reverbGroup
);
"Reverb loaded".postln;

In [ ]:
%%scv

~paddur_min = 4.5;
~paddur_max = 5.5;
~mdur_min = 0.99;
~mdur_max = 1;
~mfreq = 1;
~mdetune = 0;
~mrq_min = 0.005;
~mrq_max = 0.008;
~mcf = 1;
~matk = 3;
~msus = 1;
~mrel = 5;
~mamp = 0.9;
~mpan_min = 0;
~mpan_max = 0;

e = Dictionary.new;

e.add(\pad_sine_lf -> {
    ~chords = Pbind(
        \instrument, \bpfsine,
        \dur, Pwhite(Pfunc{~paddur_min}, Pfunc{~paddur_max}),
        \midinote, Pxrand([
            [23,35,54,63,64],
            [45,52,54,59,61,64],
            [28,40,47,56,59,63],
            [42,52,57,61,63]
        ], inf),
        \detune, Pexprand(0.05,0.2),
        \cfmin, 500,
        \cfmax, 1000,
        \rqmin, Pexprand(0.01,0.02),
        \rqmax, Pexprand(0.2,0.3),
        \atk, Pwhite(2.0,2.5),
        \rel, Pwhite(6.5,10.0),
        \ldb, 6,
        \amp, 0.3,
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).play;
    
    ~marimba = Pbind(
        \instrument, \bpfsaw,
        \dur, Pwhite(Pfunc{~mdur_min}, Pfunc{~mdur_max}),
        \freq, Prand([1/2, 2/3, 1], inf) * Pfunc{~mfreq},
        \detune, Pfunc({~mdetune}),
        \rqmin, Pfunc{~mrq_min},
        \rqmax, Pfunc{~mrq_max},
        \cfmin, Prand((Scale.major.degrees+64).midicps,inf) *
        (Prand(([1,2,4]), inf) * round((Pfunc{~mcf}))),
        \cfmax, Pkey(\cfmin) * Pwhite(1.008,1.025),
        \atk, Pfunc{~matk},
        \sus, Pfunc{~msus},
        \rel, Pfunc{~mrel},
        \amp, Pfunc{~mamp},
        \pan, Pwhite(Pfunc{~mpan_min},Pfunc{~mpan_max}),
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).play;
});
e.add(\event_stop -> {
    ~chords.stop;
    ~marimba.stop;
});
"Events loaded".postln;

In [ ]:
import ipywidgets
import os
import threading
from IPython.display import clear_output

In [ ]:
class Bloodplayer:
    
    def __init__(self, data, pulse_time=0.1, verbose=False):
        self.lock = threading.Lock()
        self.stopevent = threading.Event()
        self.callback_fn = None
        self.idx = 0
        self.data = data
        self.length = data.shape[0]
        self.verbose = verbose
        self.pulse_time = pulse_time
        self.rtime = 0
        
    #def __del__():
        # close plot window
        #pass
    
    def callback_fn_default(self, v):
        os.write(1, f"\r                       \r{v}".encode())
        
    def procfn(self):
        self.idx = 0
        self.rtime = 0
        while not self.stopevent.wait(0) and self.idx < self.length-1:
            v = self.data[self.idx]
            if self.verbose: 
                os.write(1, f"\r{self.idx}:{self.idx}                   ".encode())
            if callable(self.callback_fn):
                self.callback_fn(self)
            else:
                self.callback_fn_default(v)
            self.rtime += self.pulse_time
            self.idx = int(self.rtime)
            time.sleep(self.pulse_time)
        print("done.")
    
    def set_callback(self, fn):
        self.callback_fn = fn
        
    def create_thread(self):
        threadname = "BloodPlayer-thread"
        # check first if it already exists
        if threadname in [t.name for t in threading.enumerate()]:
            print("create_thread: thread is already existing, stop first")
        else:
            self.stopevent.clear()
            self.producer = threading.Thread(name=threadname, target=self.procfn, args=[])
            self.producer.start()

    def stop_thread(self):
        self.stopevent.set()

In [ ]:
bloodplayer = Bloodplayer(delta)

In [ ]:
# Plot Data 
%matplotlib

# create figure
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); 
mngr.window.setGeometry(1200, 0, 500, 400)

# create axis, plots
ax.clear()
plmarked, = ax.plot([], [], "r-", lw=1)
#pldata, = ax.plot(dsf[:,0], "-", ms=2) # create plots
pldata, = ax.plot(delta, "-", ms=2)
#ax.plot(dsf[:,8], color='r', lw=1)

def update_plot(t): 
    global fig, ax, plmarked, pldata
    plmarked.set_data([t,t], [-10, 10])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(plmarked)
    fig.canvas.update()

def onclick(event):
    global bloodplayer
    if event.dblclick:
        print(event.button, event.xdata)
        bloodplayer.idx = int(event.xdata)

connection_id = fig.canvas.mpl_connect('button_press_event', onclick)

# test with 
# update_plot(5000)

In [ ]:
# GUI
def start(b):
    global bloodplayer
    bloodplayer.create_thread()
    %sc e[\pad_sine_lf].value;
    print("start")
b1 = ipywidgets.Button(description='Start') 
b1.on_click(start)

def stop(b):
    global bloodplayer
    print("stop")
    bloodplayer.stop_thread()
    %sc e[\event_stop].value;

b2 = ipywidgets.Button(description='Stop') 
b2.on_click(stop)
out = ipywidgets.Output()
ipywidgets.HBox([b1, b2, out])

In [ ]:
# Custom code for sonifications

kalimba = sc.Buffer().load_file("samples/kalimba.wav")
order = 1
cf = 0.2
sr = 1/bloodplayer.pulse_time
b, a = signal.butter(order, cf, fs=sr)
zi = signal.lfilter_zi(b, a)

#pz = 0
#ppz = 0
# print(zi)

def son_waterdrop(self):
    global a, b, zi, pz, ppz, nd, nv 
    global revtime, mix, predelay, amp 
    global paddur_min, paddur_max, mamp 
    global mdur_min, mdur_max, mfreq, mdetune, mrq_min, mrq_max, mcf, matk, msus, mrel, mpan_min, mpan_max 
    
    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]
    
    # calculate filters
    z, zi = signal.lfilter(b, a, np.array([delta_val]), zi=zi)
    #print(z[0])
    
    # normalize delta and volume
    nd = scn.linlin(delta_val, delta_min, delta_max, 0, 1)       
    nv = scn.linlin(volume_val, volume_min, volume_max, 0, 1)
    
    # set reverb based on normalized values of delta and volume
    revtime = scn.linlin(nv, 0, 1, 1.8, 0.5)
    mix = scn.linlin(nv, 0, 1, 0.5, 0.1)
    predelay = scn.linlin(nv, 0, 1, 0.4, 0.1)
    amp = scn.linlin(nv, 0, 1, 0.8, 0.2)        
    paddur_min = 4.5-(scn.linlin(nd,0,1,0,4))    
    paddur_max = 5.5-(scn.linlin(nd,0,1,0,4))
    mdur_min = scn.linlin(nd,0,1,0.99,0.05)
    mdur_max = scn.linlin(nd,0,1,1,0.1)
    mfreq = scn.linlin(nd,0,1,1,4)
    mdetune = scn.linlin(nd,0,1,0,2)
    mrq_min = scn.linlin(nv,0,1,0.005,0.09)
    mrq_max = scn.linlin(nv,0,1,0.008,0.2)
    mcf = scn.linlin(nd,0,1,1,5)
    matk = scn.linlin(nv,0,1,3,1.5)
    msus = scn.linlin(nv,0,1,1,0.5)
    mrel = scn.linlin(nv,0,1,5,2.5)
    mamp = scn.linlin(nd,0,0.1,0.2,0.8) 
    mpan_min = scn.linlin(nd,0,1,0,-1)
    mpan_max = scn.linlin(nd,0,1,0,1)

    %sc ~reverbSynth.set(\revtime, ^revtime, \mix, ^mix, \predelay, ^predelay, \amp, ^amp)        
    %sc ~paddur_min = ^paddur_min
    %sc ~paddur_max = ^paddur_max
    %sc ~mdur_min = ^mdur_min
    %sc ~mdur_max = ^mdur_max
    %sc ~mfreq = ^mfreq
    %sc ~mdetune = ^mdetune
    %sc ~mrq_min = ^mrq_min
    %sc ~mrq_max = ^mrq_max
    %sc ~mcf = ^mcf
    %sc ~matk = ^matk
    %sc ~msus = ^msus
    %sc ~mrel = ^mrel
    %sc ~mamp = ^mamp
    %sc ~mpan_min = ^mpan_min
    %sc ~mpan_max = ^mpan_max
    
    os.write(1, f"\r{float(self.rtime):9.5}, {self.idx}, {float(z):9.7} :callback{self.idx}                   ".encode())

    #if np.argmax([z[0],pz,ppz]) == 1:
        #   print("pz is max")
        ## sonify pz
    
    #ppz = pz
    #pz = z[0]
    
    update_plot(self.idx)

bloodplayer.set_callback(son_waterdrop)

In [ ]:
#bloodplayer.pulse_time = 0.1
#bloodplayer.pulse_time = 1